In [1]:
import os
import re
import cv2
import numpy as np
import csv
from datetime import date,datetime
import math
import pandas as pd
from PIL import Image
import smtplib

studentlabels={'3019' : 'Harini' ,'3022' : 'Jumana Begum','3026' : 'Karthika' ,'3030' : 'Kirupa' ,'3039' : 'Narmatha' ,'3047' : 'Preethi','3049' : 'Ramya','3055' : 'Samreen Ayesha','3063' : 'Sineka'  }
now = datetime.now()
today830am = now.replace(hour=15, minute=0, second=0, microsecond=0)
today900am = now.replace(hour=16, minute=0, second=0, microsecond=0)

In [14]:
#training the LBPH model for front
faces = []
labels = []

path = "C:/Users/Samreen/CIP/trainFront_new"
dir_list = os.listdir(path)

for elt in dir_list:
    impath = path+'/'+elt
    image = cv2.imread(impath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mystring = impath.split("/")[5]
    mystring = mystring.split("(")[0]
    labels.append(int(mystring))
    faces.append(gray)
    
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

face_recognizer_front = cv2.face.LBPHFaceRecognizer_create(radius=5, neighbors=4, grid_x=8, grid_y=8)
face_recognizer_front.train(faces, np.array(labels))

#predicting the student
def predict_front(test_img):
    #make a copy of the image as we don't want to chang original image
    
    img = test_img.copy()
    label= face_recognizer_front.predict(img)

    label_text = studentlabels[str(label[0])]
    score = label[1]
    score = math.ceil(score)
    
    return label_text,score

Total faces:  90
Total labels:  90


In [15]:
#training the LBPH model for left side
faces = []
labels = []

path = "C:/Users/Samreen/CIP/train_left_side_new"
dir_list = os.listdir(path)

for elt in dir_list:
    impath = path+'/'+elt
    image = cv2.imread(impath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mystring = impath.split("/")[5]
    mystring = mystring.split("(")[0]
    labels.append(int(mystring))
    faces.append(gray)
    
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

face_recognizer_left_side = cv2.face.LBPHFaceRecognizer_create(radius=5, neighbors=4, grid_x=8, grid_y=8)
face_recognizer_left_side.train(faces, np.array(labels))

#predicting the student
def predict_left_side(test_img):
    #make a copy of the image as we don't want to chang original image
    
    img = test_img.copy()
    label= face_recognizer_left_side.predict(img)

    label_text = studentlabels[str(label[0])]
    score = label[1]
    score = math.ceil(score)
    
    return label_text,score

Total faces:  90
Total labels:  90


In [16]:
#training the LBPH model for right side
faces = []
labels = []

path = "C:/Users/Samreen/CIP/train_right_side_new"
dir_list = os.listdir(path)

for elt in dir_list:
    impath = path+'/'+elt
    image = cv2.imread(impath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mystring = impath.split("/")[5]
    mystring = mystring.split("(")[0]
    labels.append(int(mystring))
    faces.append(gray)
    
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

face_recognizer_right_side = cv2.face.LBPHFaceRecognizer_create(radius=5, neighbors=4, grid_x=8, grid_y=8)
face_recognizer_right_side.train(faces, np.array(labels))

#predicting the student
def predict_right_side(test_img):
    #make a copy of the image as we don't want to chang original image
    
    img = test_img.copy()
    label= face_recognizer_right_side.predict(img)

    label_text = studentlabels[str(label[0])]
    score = label[1]
    score = math.ceil(score)
    
    return label_text,score

Total faces:  90
Total labels:  90


In [6]:
#csv file creation for the day
def createfile():
    i = 0
    students={'2019103019' : ['Harini','harinisasi26@gmail.com'] ,
              '2019103022' : ['Jumana Begum','begumjumana2712@gmail.com'],
              '2019103026' : ['Karthika','karthikat008@gmail.com'],
              '2019103030' : ['Kirupa','kirupashrir2001@gmail.com'],
              '2019103039' : ['Narmatha','narmathama23@gmail.com'],
              '2019103047' : ['Preethi','preethiba@gmail.com'],
              '2019103049' : ['Ramya','ramya26@gmail.com'],
              '2019103055' : ['Samreen Ayesha','samreenayesha137@gmail.com'],
              '2019103063' : ['Sineka','sinekapkt@gmail.com']}
    header = ['Name','Register No','Email ID','Time of entry','Status']
    today = date.today()
    current_date = today.strftime("%b-%d-%Y")
    f = open(current_date+'.csv', 'w',newline='')
    writer = csv.writer(f)
    writer.writerow(header)
    Empt= ''
    abse = 'Absent'
    for key in students.keys():
            f.write("%s,%s,%s,%s,%s\n" % (students[key][0], key,students[key][1],Empt,abse))
            i += 1

    f.close()


In [7]:
def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()


In [8]:
def attendanceupdate(label_text,score):
    today = date.today()
    current_date = today.strftime("%b-%d-%Y")
    df = pd.read_csv(current_date+'.csv')  
    csv_file = csv.reader(open(current_date+'.csv', "r"), delimiter=",")
    
    if score <= 20:
        i = -2
        for row in csv_file:
            i += 1
            if label_text==row[0]:
                now = datetime.now()
                nowstr = now.strftime("%H:%M:%S")
                if now < today830am  and row[4]=="Absent":
                    df.loc[i, "Status"] = 'Present'
                    df.loc[i,'Time of entry'] = nowstr
                elif now < today900am and row[4]=="Absent":
                    df.loc[i, "Status"] = 'Late'
                    df.loc[i,'Time of entry'] = nowstr
                df.to_csv(current_date+'.csv', index=False)

In [10]:
def sendmail():
    today = date.today()
    current_date = today.strftime("%b-%d-%Y")
    csv_file = csv.reader(open(current_date+'.csv', "r"), delimiter=",")
    mailabs = []
    maillate = []
    for row in csv_file:
        if row[4]=="Absent":
            mailabs.append(row[2])
        elif row[4]=="Late":
            maillate.append(row[2])
            
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login("projectmail2712@gmail.com", "ckjl zmhm otmk tqdc")
    for dest in mailabs:
        message = "Your ward was absent today."
        s.sendmail("projectmail2712@gmail.com",dest,message)
    for dest in maillate:
        message = "Your ward was late today."
        s.sendmail("projectmail2712@gmail.com",dest,message)
    s.quit()

In [11]:
def sendmailintruder():
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders

    fromaddr = "projectmail2712@gmail.com"
    toaddr = "begumjumana2712@gmail.com"

    msg = MIMEMultipart()
    msg['From'] = "projectmail2712@gmail.com"
    msg['To'] = "begumjumana2712@gmail.com"
    msg['Subject'] = "INTRUDER LIST"
    body = "IMAGES OF THE INTRUDER"
    msg.attach(MIMEText(body, 'plain'))
    
    #looping through the intruders images
    path = "C:/Users/Samreen/CIP/INTRUDERS"
    dir_list = os.listdir(path)

    for elt in dir_list:
        filename = elt
        impath = path+'/'+elt
        attachment = open(impath, "rb")
        p = MIMEBase('application', 'octet-stream')
        p.set_payload((attachment).read())
        encoders.encode_base64(p)
        p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
        msg.attach(p)

    #sending email to teacher regarding the intruders
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login("projectmail2712@gmail.com", "ckjl zmhm otmk tqdc")
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    s.quit()


In [21]:
#driver program

createfile()
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
sideCascade = cv2.CascadeClassifier("haarcascade_profileface.xml")

cap = cv2.VideoCapture(0)  
if (cap.isOpened()== False):
  print("Error opening video stream or file")
i = 0
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    #gray scaling
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    #clahe
    clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
    gray_frame = clahe.apply(gray_frame)
    
    faces1 = faceCascade.detectMultiScale(
        gray_frame,
        scaleFactor = 1.05,
        minNeighbors = 20,
        minSize = (100,100),
    )
    faces2 = sideCascade.detectMultiScale(
        gray_frame,
        scaleFactor = 1.05,
        minNeighbors = 20,
        minSize = (100,100),
    )
  
    height = 500
    width = 500

    for(x,y,w,h) in faces1:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),4)
        #crop
        crop_img1 = gray_frame[y:y+h, x:x+w]
        #resize
        crop_img1 = cv2.resize(crop_img1, (width, height))
        #sharpen
        if(variance_of_laplacian(crop_img1) <= 5):
            print("discarded : "+str(variance_of_laplacian(crop_img1)))
            continue
        elif(variance_of_laplacian(crop_img1) <= 200):
            kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
            image = cv2.filter2D(src=crop_img1, ddepth=-1, kernel=kernel)
        #predict
        label_text1,score1 = predict_front(image)
        label_text2,score2 = predict_left_side(image)
        label_text3,score3 = predict_right_side(image)
        
        if score1<score2 and score1<score3:
            label_text = label_text1
            score = score1
        elif score2<score1 and score1<score3:
            label_text = label_text2
            score = score2
        else:
            label_text = label_text3
            score = score3
        print(label_text,score)

        if score >= 30 :
            cv2.imwrite("CIP/INTRUDERS/intruder"+str(i)+".jpeg",image)
            i += 1
        attendanceupdate(label_text,score)
        
        
    for(x,y,w,h) in faces2:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),4)
        #crop
        crop_img2 = gray_frame[y:y+h, x:x+w]
        #resize
        crop_img2 = cv2.resize(crop_img2, (width, height))
        #sharpen
        if(variance_of_laplacian(crop_img2) <= 10):
            print("discarded : "+str(variance_of_laplacian(crop_img2)))
            continue
        elif(variance_of_laplacian(crop_img2) <= 100):
            kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
            image = cv2.filter2D(src=crop_img2, ddepth=-1, kernel=kernel)
        #predict
        label_text1,score1 = predict_front(image)
        label_text2,score2 = predict_left_side(image)
        label_text3,score3 = predict_right_side(image)
        
        if score1<score2 and score1<score3:
            label_text = label_text1
            score = score1
        elif score2<score1 and score2<score3:
            label_text = label_text2
            score = score2
        else:
            label_text = label_text3
            score = score3
            
        print(label_text,score)
       
        #intruders
        if score >= 30 :
            cv2.imwrite("CIP/INTRUDERS/intruder"+str(i)+".jpeg",image)
            i += 1
        attendanceupdate(label_text,score)
    
    cv2.imshow('Frame',frame)
    if cv2.waitKey(1000) & 0xFF == ord('q'):
        break
  else:
        break
cap.release()
cv2.destroyAllWindows()

sendmail()
sendmailintruder()


Ramya 102
Ramya 106
Priya 105
Ramya 104
Ramya 104
Jumana Begum 105
Priya 108
Priya 107
Priya 106
Ramya 105
Ramya 105


In [44]:
#ACCURACY FOR TEST DATASET WITHOUT CLAHE AND WITHOUT BLUR
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
path = "C:/Users/Samreen/CIP/test"
test_labels = ['Harini','Harini','Narmatha', 'Narmatha', 'Kirupa', 'Kirupa', 'intruder', 'intruder', 'Preethi', 'Samreen Ayesha', 'Samreen Ayesha', 'intruder', 'intruder', 'Samreen Ayesha', 'Samreen Ayesha', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Ramya', 'Ramya', 'Ramya', 'Sineka', 'Samreen Ayesha', 'Samreen Ayesha', 'Samreen Ayesha', 'Karthika', 'Karthika', 'Preethi', 'Jumana Begum', 'Jumana Begum']
#in lexicographical order of file
test_pred = []
dir_list = os.listdir(path)
i = 0
for elt in dir_list:
    impath = path+'/'+elt
    print(impath)
    image = cv2.imread(impath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    label_text3,score3 = predict_front(image)
    label_text1,score1 = predict_left_side(image)
    label_text2,score2 = predict_right_side(image)
    print(label_text3,score3)
    print(label_text1,score1)
    print(label_text2,score2)
    if score1<score2 and score1<score3:
        label_text = label_text1
        score = score1
    elif score2<score1 and score2<score3:
        label_text = label_text2
        score = score2
    else:
        label_text = label_text3
        score = score3 
    print(label_text,score)
    print("ACTUAL:")
    print(test_labels[i])
    #test_pred.append(label_text)
    if(score>=30):
        test_pred.append("intruder")
        print("Intruder")
        #Coz in our test dataset all members are present and any intruder will be considered wrong '''
    else:
        test_pred.append(label_text)
    i = i + 1
print(test_pred)
print(test_labels)
print("Accuracy Score: ", accuracy_score(test_labels, test_pred))
print("Precision Score: ", precision_score(test_labels, test_pred,average='macro'))
print("Recall Score: ", recall_score(test_labels, test_pred,average='macro'))
print("F1 Score: ",f1_score(test_labels, test_pred,average='macro'))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_pred))
   
   

C:/Users/Samreen/CIP/test/30 (11).jpeg
Ramya 65
Narmatha 78
Karthika 69
Ramya 65
ACTUAL:
Harini
Intruder
C:/Users/Samreen/CIP/test/30 (12).jpeg
Harini 31
Harini 52
Karthika 48
Harini 31
ACTUAL:
Harini
Intruder
C:/Users/Samreen/CIP/test/crop280.jpeg
Narmatha 32
Sineka 64
Karthika 57
Narmatha 32
ACTUAL:
Narmatha
Intruder
C:/Users/Samreen/CIP/test/crop490.jpeg
Harini 55
Karthika 59
Narmatha 37
Narmatha 37
ACTUAL:
Narmatha
Intruder
C:/Users/Samreen/CIP/test/crop600.jpeg
Kirupa 36
Sineka 55
Karthika 51
Kirupa 36
ACTUAL:
Kirupa
Intruder
C:/Users/Samreen/CIP/test/crop650.jpeg
Kirupa 38
Sineka 57
Karthika 53
Kirupa 38
ACTUAL:
Kirupa
Intruder
C:/Users/Samreen/CIP/test/crop900.jpeg
Ramya 43
Sineka 60
Karthika 53
Ramya 43
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/crop940.jpeg
Ramya 44
Sineka 60
Karthika 53
Ramya 44
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/Test (1).jpeg
Preethi 26
Sineka 43
Karthika 29
Preethi 26
ACTUAL:
Preethi
C:/Users/Samreen/CIP/test/Test (18).jpeg
Samreen

C:\Users\Samreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
#ACCURACY FOR TEST DATASET WITH CLAHE AND WITHOUT BLUR
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
path = "C:/Users/Samreen/CIP/test"
test_labels = ['Harini','Harini','Narmatha', 'Narmatha', 'Kirupa', 'Kirupa', 'intruder', 'intruder', 'Preethi', 'Samreen Ayesha', 'Samreen Ayesha', 'intruder', 'intruder', 'Samreen Ayesha', 'Samreen Ayesha', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Ramya', 'Ramya', 'Ramya', 'Sineka', 'Samreen Ayesha', 'Samreen Ayesha', 'Samreen Ayesha', 'Karthika', 'Karthika', 'Preethi', 'Jumana Begum', 'Jumana Begum']
#in lexicographical order of file
test_pred = []
dir_list = os.listdir(path)
i = 0
for elt in dir_list:
    impath = path+'/'+elt
    print(impath)
    image = cv2.imread(impath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #clahe
    clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
    image = clahe.apply(image)
    
    label_text3,score3 = predict_front(image)
    label_text1,score1 = predict_left_side(image)
    label_text2,score2 = predict_right_side(image)
    print(label_text3,score3)
    print(label_text1,score1)
    print(label_text2,score2)
    
    if score1<score2 and score1<score3:
        label_text = label_text1
        score = score1
    elif score2<score1 and score2<score3:
        label_text = label_text2
        score = score2
    else:
        label_text = label_text3
        score = score3 
    print(label_text,score)
    print("ACTUAL:")
    print(test_labels[i])
    #test_pred.append(label_text)
    if(score>=30):
        test_pred.append("intruder")
        print("Intruder")
        #Coz in our test dataset all members are present and any intruder will be considered wrong '''
    else:
        test_pred.append(label_text)
    i = i + 1
print(test_pred)
print(test_labels)
print("Accuracy Score: ", accuracy_score(test_labels, test_pred))
print("Precision Score: ", precision_score(test_labels, test_pred,average='macro'))
print("Recall Score: ", recall_score(test_labels, test_pred,average='macro'))
print("F1 Score: ",f1_score(test_labels, test_pred,average='macro'))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_pred))

C:/Users/Samreen/CIP/test/30 (11).jpeg
Ramya 50
Sineka 62
Karthika 53
Ramya 50
ACTUAL:
Harini
Intruder
C:/Users/Samreen/CIP/test/30 (12).jpeg
Harini 16
Sineka 38
Karthika 35
Harini 16
ACTUAL:
Harini
C:/Users/Samreen/CIP/test/crop280.jpeg
Narmatha 19
Sineka 49
Karthika 45
Narmatha 19
ACTUAL:
Narmatha
C:/Users/Samreen/CIP/test/crop490.jpeg
Harini 43
Karthika 44
Narmatha 24
Narmatha 24
ACTUAL:
Narmatha
C:/Users/Samreen/CIP/test/crop600.jpeg
Kirupa 23
Sineka 43
Ramya 39
Kirupa 23
ACTUAL:
Kirupa
C:/Users/Samreen/CIP/test/crop650.jpeg
Kirupa 26
Sineka 44
Ramya 41
Kirupa 26
ACTUAL:
Kirupa
C:/Users/Samreen/CIP/test/crop900.jpeg
Ramya 33
Sineka 48
Karthika 43
Ramya 33
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/crop940.jpeg
Ramya 34
Sineka 47
Karthika 42
Ramya 34
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/Test (1).jpeg
Preethi 20
Sineka 34
Karthika 22
Preethi 20
ACTUAL:
Preethi
C:/Users/Samreen/CIP/test/Test (18).jpeg
Samreen Ayesha 29
Sineka 38
Karthika 34
Samreen Ayesha 29
AC

In [46]:
#ACCURACY FOR TEST DATASET WITHOUT CLAHE AND WITH BLUR
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
path = "C:/Users/Samreen/CIP/test"
test_labels = ['Harini','Harini','Narmatha', 'Narmatha', 'Kirupa', 'Kirupa', 'intruder', 'intruder', 'Preethi', 'Samreen Ayesha', 'Samreen Ayesha', 'intruder', 'intruder', 'Samreen Ayesha', 'Samreen Ayesha', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Ramya', 'Ramya', 'Ramya', 'Sineka', 'Samreen Ayesha', 'Samreen Ayesha', 'Samreen Ayesha', 'Karthika', 'Karthika', 'Preethi', 'Jumana Begum', 'Jumana Begum']
#in lexicographical order of file
test_pred = []
dir_list = os.listdir(path)
i = 0
for elt in dir_list:
    impath = path+'/'+elt
    print(impath)
    image = cv2.imread(impath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if(variance_of_laplacian(image) <= 5):
        print(i)
        del test_labels[i]
        continue
    elif(variance_of_laplacian(image) <= 200):
        print("Cleared")
        kernel = np.array([[0, -1, 0],
                    [-1, 5,-1],
                    [0, -1, 0]])
        image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    
    label_text3,score3 = predict_front(image)
    label_text1,score1 = predict_left_side(image)
    label_text2,score2 = predict_right_side(image)
    print(label_text3,score3)
    print(label_text1,score1)
    print(label_text2,score2)
    if score1<score2 and score1<score3:
        label_text = label_text1
        score = score1
    elif score2<score1 and score2<score3:
        label_text = label_text2
        score = score2
    else:
        label_text = label_text3
        score = score3 
    print(label_text,score)
    print("ACTUAL:")
    print(test_labels[i])
    #test_pred.append(label_text)
    if(score>=30):
        test_pred.append("intruder")
        print("Intruder")
        #Coz in our test dataset all members are present and any intruder will be considered wrong '''
    else:
        test_pred.append(label_text)
    i = i + 1
print(test_pred)
print(test_labels)
print("Accuracy Score: ", accuracy_score(test_labels, test_pred))
print("Precision Score: ", precision_score(test_labels, test_pred,average='macro'))
print("Recall Score: ", recall_score(test_labels, test_pred,average='macro'))
print("F1 Score: ",f1_score(test_labels, test_pred,average='macro'))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_pred))

C:/Users/Samreen/CIP/test/30 (11).jpeg
Ramya 65
Narmatha 78
Karthika 69
Ramya 65
ACTUAL:
Harini
Intruder
C:/Users/Samreen/CIP/test/30 (12).jpeg
1
C:/Users/Samreen/CIP/test/crop280.jpeg
Cleared
Narmatha 8
Sineka 34
Karthika 30
Narmatha 8
ACTUAL:
Narmatha
C:/Users/Samreen/CIP/test/crop490.jpeg
2
C:/Users/Samreen/CIP/test/crop600.jpeg
Cleared
Kirupa 11
Sineka 30
Preethi 27
Kirupa 11
ACTUAL:
Kirupa
C:/Users/Samreen/CIP/test/crop650.jpeg
Cleared
Kirupa 12
Sineka 30
Jumana Begum 27
Kirupa 12
ACTUAL:
Kirupa
C:/Users/Samreen/CIP/test/crop900.jpeg
Ramya 43
Sineka 60
Karthika 53
Ramya 43
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/crop940.jpeg
Ramya 44
Sineka 60
Karthika 53
Ramya 44
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/Test (1).jpeg
Cleared
Preethi 13
Sineka 26
Preethi 13
Preethi 13
ACTUAL:
Preethi
C:/Users/Samreen/CIP/test/Test (18).jpeg
Samreen Ayesha 38
Sineka 47
Karthika 42
Samreen Ayesha 38
ACTUAL:
Samreen Ayesha
Intruder
C:/Users/Samreen/CIP/test/Test (19).jpeg
Samre

C:\Users\Samreen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
#ACCURACY FOR TEST DATASET WITH CLAHE AND WITH BLUR
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
path = "C:/Users/Samreen/CIP/test"
test_labels = ['Harini','Harini','Narmatha', 'Narmatha', 'Kirupa', 'Kirupa', 'intruder', 'intruder', 'Preethi', 'Samreen Ayesha', 'Samreen Ayesha', 'intruder', 'intruder', 'Samreen Ayesha', 'Samreen Ayesha', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Jumana Begum', 'Ramya', 'Ramya', 'Ramya', 'Sineka', 'Samreen Ayesha', 'Samreen Ayesha', 'Samreen Ayesha', 'Karthika', 'Karthika', 'Preethi', 'Jumana Begum', 'Jumana Begum']
#in lexicographical order of file
test_pred = []
dir_list = os.listdir(path)
i = 0
for elt in dir_list:
    impath = path+'/'+elt
    print(impath)
    image = cv2.imread(impath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #clahe
    clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
    image = clahe.apply(image)
    #blur
    if(variance_of_laplacian(image) <= 5):
        print(i)
        del test_labels[i]
        continue
    elif(variance_of_laplacian(image) <= 200):
        kernel = np.array([[0, -1, 0],
                    [-1, 5,-1],
                    [0, -1, 0]])
        image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    
    label_text3,score3 = predict_front(image)
    label_text1,score1 = predict_left_side(image)
    label_text2,score2 = predict_right_side(image)
    print(label_text3,score3)
    print(label_text1,score1)
    print(label_text2,score2)
    if score1<score2 and score1<score3:
        label_text = label_text1
        score = score1
    elif score2<score1 and score2<score3:
        label_text = label_text2
        score = score2
    else:
        label_text = label_text3
        score = score3 
    print(label_text,score)
    print("ACTUAL:")
    print(test_labels[i])
    #test_pred.append(label_text)
    if(score>=30):
        test_pred.append("intruder")
        print("Intruder")
        #Coz in our test dataset all members are present and any intruder will be considered wrong '''
    else:
        test_pred.append(label_text)
    i = i + 1
print(test_pred)
print(test_labels)
print("Accuracy Score: ", accuracy_score(test_labels, test_pred))
print("Precision Score: ", precision_score(test_labels, test_pred,average='macro'))
print("Recall Score: ", recall_score(test_labels, test_pred,average='macro'))
print("F1 Score: ",f1_score(test_labels, test_pred,average='macro'))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_pred))

C:/Users/Samreen/CIP/test/30 (11).jpeg
Ramya 50
Sineka 62
Karthika 53
Ramya 50
ACTUAL:
Harini
Intruder
C:/Users/Samreen/CIP/test/30 (12).jpeg
Harini 3
Sineka 21
Preethi 18
Harini 3
ACTUAL:
Harini
C:/Users/Samreen/CIP/test/crop280.jpeg
Narmatha 3
Sineka 26
Karthika 23
Narmatha 3
ACTUAL:
Narmatha
C:/Users/Samreen/CIP/test/crop490.jpeg
Harini 24
Ramya 25
Narmatha 8
Narmatha 8
ACTUAL:
Narmatha
C:/Users/Samreen/CIP/test/crop600.jpeg
Kirupa 5
Sineka 23
Preethi 20
Kirupa 5
ACTUAL:
Kirupa
C:/Users/Samreen/CIP/test/crop650.jpeg
Kirupa 7
Sineka 24
Preethi 20
Kirupa 7
ACTUAL:
Kirupa
C:/Users/Samreen/CIP/test/crop900.jpeg
Ramya 33
Sineka 48
Karthika 43
Ramya 33
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/crop940.jpeg
Ramya 34
Sineka 47
Karthika 42
Ramya 34
ACTUAL:
intruder
Intruder
C:/Users/Samreen/CIP/test/Test (1).jpeg
Preethi 10
Sineka 19
Preethi 9
Preethi 9
ACTUAL:
Preethi
C:/Users/Samreen/CIP/test/Test (18).jpeg
Samreen Ayesha 29
Sineka 38
Karthika 34
Samreen Ayesha 29
ACTUAL:
Samreen